In [1]:
import spacy
nlp = spacy.load("en_core_web_md")

In [2]:
import pdb
import os
import json

from spacy.matcher import Matcher 
from spacy.tokens import Span
from spacy import displacy

In [3]:
def read_OMCS_file(filename, language='en', save_filename=None):
    '''
    Given the filename and the language retrieves the sentences from the
    provided language and stores it in another file.
    input:
        filename : string containing the name of the file
        language : string containing the name of the language
                   to look for.
    
    output: List of sentences from the OMCS text file belonging 
            to that particular language.
    '''
    data_list_language = []
    
    assert os.path.isfile(filename), 'File does not exist!'
    
    with open(filename, 'r') as f:
         data_list = f.readlines()
   
    for data_line in data_list:
        try:
            if data_line.split('\t')[4]==language:
                data_list_language.append(data_line.split('\t')[1])
        except:
            print('Found a faulty sentence :', data_line)
            
    if save_filename:
        
        with open(save_filename, 'w') as fp:
            json.dump(data_list_language, fp)
            
    return data_list_language
    
    
    

In [5]:
data = read_OMCS_file('./data/omcs-sentences-free.txt', save_filename='english_sentences.txt')


Found a faulty sentence : 1531350	Um(a) golfinho é usado(a) para brinc

Found a faulty sentence : 1194903	Some people like to check their e-mail first thing in the morning.

Found a faulty sentence : 1194907	People take showers in the morning.

Found a faulty sentence : 1194921	People eat cereal for breakfast.

Found a faulty sentence : 1194929	People can travel from work to home on a bicycle.

Found a faulty sentence : 1194960	Paper clips hold sheets of paper together.

Found a faulty sentence : 1044642	The statement "Telescopes make things look larger." is true because Refracting telescopes use lenses to gather and bend

Found a faulty sentence : (898159 rows)



In [4]:
with open('english_sentences.txt', 'r') as fp:
    en_data = json.load(fp)

In [4]:
#printing token information using spacy
sentence = en_data[121]
tag_info = nlp(sentence)
for token in tag_info:
    print(token.text, token.tag_, token.dep_, token.head.text, token.head.pos_)

Sometimes RB advmod causes VERB
lightning NN nsubj causes VERB
causes VBZ ROOT causes VERB
electricity NN compound shortouts NOUN
shortouts NNS dobj causes VERB


In [6]:
#getting noun chunks using spacy
doc = nlp(sentence)
for chunk in doc.noun_chunks:
    print(chunk.text, " | ", chunk.root.text, " | ", chunk.root.dep_, " | ",
            chunk.root.head.text)

lightning  |  lightning  |  nsubj  |  causes
electricity shortouts  |  shortouts  |  dobj  |  causes


In [7]:
#navigating the parse tree
sentence = "the film had 200 patents."
doc = nlp(sentence)
for token in doc:
    print(token.text," | ",  token.dep_," | ",  token.head.text," | ",  token.head.pos_," | ", 
            [child.text for child in token.children])

the  |  det  |  film  |  NOUN  |  []
film  |  nsubj  |  had  |  AUX  |  ['the']
had  |  ROOT  |  had  |  AUX  |  ['film', 'patents', '.']
200  |  nummod  |  patents  |  NOUN  |  []
patents  |  dobj  |  had  |  AUX  |  ['200']
.  |  punct  |  had  |  AUX  |  []


In [5]:
#code for getting the entities and relations from the tutorial

def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]


def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1", None, pattern) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [6]:
#my code for extracting entities and relations

def extract_subject_object(sent):
    doc = nlp(sent)
    subject = []
    obj = []
    for token in doc:
        
        if token.dep_.find("subj") == True and len(subject) == 0:
            for child in token.children:
                if child.dep_ not in ["det", "punct", "conj", "prep"]:
                    subject.append(child.text)
            subject.append(token.text)
        
        if token.dep_.find("obj") == True and len(obj) == 0:
            for child in token.children:
                if child.dep_ not in ["det", "punct", "conj", "prep"]:
                    obj.append(child.text)
            obj.append(token.text)
    
    subject_string = ""
    for token in subject:
        subject_string += token
        subject_string += " "
    sub_string = subject_string.strip()
    
    object_string = ""
    for token in obj:
        object_string += token
        object_string += " "
    obj_string = object_string.strip(" ")
    return (sub_string, obj_string)


def extract_entities_from_clause(clause):
    '''
    Tring someway to traverse the tree from the root and try finding 
    relationships from there onwards
    '''
    doc = nlp(clause)
    root_children = []
    subject = []
    object_entity = ''
    subject_entity = ''
    for token in doc:
        
        if token.dep_.find("obj") == True:
            object_entity = get_subtree_excluding_subclause(token)
            print(" The object : ",get_subtree_excluding_subclause(token))

        if token.dep_.find("subj") == True:
            subject_entity = get_subtree_excluding_subclause(token)
            print(" The subject :", get_subtree_excluding_subclause(token))
    
    return subject_entity, object_entity


def extract_relation_from_clause(clause):
    '''
    Given a clause (independent or dependent) returns the 
    possible relation (verb). The relation is calculated 
    by taking the verb and 
    
    
    input:
        clause : a string containing a single clause
    '''
    doc = nlp(clause)
    dep_list = ['ROOT']
    for token in doc:
        if token.dep_ in dep_list:
            return token
            
def extract_entity_relation_tuples(sent):
    '''
    Given a sentence, returns a list of entity relation tuples
    Entity relation tuples are extracted for each of the clauses
    detected in the sentence.
    input:
        sent : A sentence in the form of a string.
    
    output:
        
        entity_rel_list : A list containing enitiy relations extracted from 
                          individual clause.
                          of the form [((subj, obj), rel), (), ()]
    '''
    clause_list = extract_clauses(sent)
    entity_rel_list = []
    for clause in clause_list:
        
        subj, obj = extract_entities_from_clause(clause)
        rel = extract_relation_from_clause(clause)
        entity_rel_list.append(((subj, obj), rel))
    
    
    return entity_rel_list
    


In [7]:
#my code for extracting entities and relations

    
def check_verb_subject_in_clause(clause_token_list):
    '''
    Given a clause/sentence checks if the sentence contains
    a subject and a verb.
    input:
        clause : a string containing a sentence
        
    output:
        contains : A boolean flag indicating the presence 
                   of a verb and a subject in the sentence.
    '''
    pos_list = ["VERB"]
    dep_list = ['aux']
    contains = False
    contains_verb = False
    contains_sub = False
    for token in clause_token_list:
        print("token : {} | pos : {} | dep : {}".format(token.text, token.pos_, token.dep_))
        if token.dep_.find("subj") == True:
            contains_sub = True
        if token.pos_ in pos_list or token.dep_ in dep_list:
            contains_verb = True
        print("Contains verb : {}, contains sub : {}".format(contains_verb, contains_sub))
    if contains_verb and contains_sub:
        contains = True
    return contains
    
         
def get_subtree_excluding_subclause2(root_token, token_subtree, clausal_list):
    '''
    Gets  the subtree excluding "relevant" subclause. To qualify
    for a clause, the part of the sentence should contain a 
    subject and a verb
    '''
    subtree_wo_clause = []
    subtree_wo_clause_string = ""
    accessed_token_list = []
    for token in token_subtree:
        print("TOKEN :", token.text)
        if token not in accessed_token_list:
            if token.dep_ in clausal_list:
                print("Calling function using token :", token.text)
                pdb.set_trace()
                token_list, subtree_string = get_subtree_excluding_subclause2(token.subtree, clausal_list)
                for tok in token_list:
                    accessed_token_list.append(tok)
                if len(token_list) == 0: #invalid subclause
                    subtree_wo_clause.append(token)
            else:
                subtree_wo_clause.append(token)
    
    #once the entire token_subtree is scanned and collected for
    #relevant tokens, check if it has a subject and a verb.
    #if it does, it qualifies as a clause, else return empty list
    #empty string
    if check_verb_subject_in_clause(subtree_wo_clause):
        for word in subtree_wo_clause:
            subtree_wo_clause_string += word.text
            subtree_wo_clause_string += " "

    subtree_wo_clause_string = subtree_wo_clause_string.strip()            
                
                
    
        
def get_subtree_excluding_subclause(node):
    '''
    Given the node, returns the subtree rooted at that node:
    ideally to extract the subject or object subtree.
    '''
    black_list = ["det", "punct", "conj", "cconj"]
    clausal_list = ["csubj", 'ccomp', 
                    'advcl', 'acl', 'conj',
                    'relcl']
    entity = []
    entity_string = ""
    for left in node.lefts:
        
        subtree_tokens, subtree_string, dep = get_subtree_excluding_subclause(left)
        if dep not in clausal_list and check_verb_subject_in_clause(subtree_tokens):
            for tok in subtree_tokens:
                entity.append(tok)
            
    entity.append(node)
    
    for right in node.rights:
        
        subtree_tokens, subtree_string, dep = get_subtree_excluding_subclause(right)
        if dep not in clausal_list and check_verb_subject_in_clause(subtree_tokens):
            for tok in subtree_tokens:
                entity.append(tok)
    
    for value in entity:
        entity_string += value.text
        entity_string += " "
        
    entity_string = entity_string.strip()
    return entity, entity_string, node.dep_
    
def extract_clauses(sent):
    '''
    Given a sentence, will break the sentence into 
    different clauses
    '''
    doc = nlp(sent)
    #the idea is to find one of the dep_ from the following 
    #list 
    '''
    [ parataxis, 
      conj - conjunct,
      ccomp - clausal complement,
      advcl - adverbal complement,
      relcl - relative clause modifier,
      csubj - clausal subject
      xcomp - open clausal complement]
    '''
    clause_dep_list = ["ROOT", "csubj", 'ccomp', 
                    'advcl', 'acl', 'conj',
                   'relcl']
    
    clause_list = []
    for token in doc:
        
        if token.dep_ in clause_dep_list:
            
            subclause, _ = get_subtree_excluding_subclause(token.subtree)
            #check if the subclause contains a verb and a subject
            #if not then it does not qualify as a clause
            check_verb_subject_in_clause(subclause)
            clause_list.append(subclause)
    
    return clause_list



In [ ]:
sentence = en_data[88120]
sentence1 = "the city of the loir-et-cher is part of the greater mumbai"
sentence2 = 'Even with the weather being that nasty, the couple and their families decided to go ahead \
with the wedding as planned.'
sentence3 = 'My little daughter loves to play with her dolls.'
sentence4 = 'Because she loves her students, Mrs Stevens will be sad on the last day of school.'
compound_sentence = 'I like road bikes, and he likes mountain bikes.'
compound_sentence = 'She ran with the dogs, I swam with the fishes, and they biked to the mountains.'
sentence5 = 'My mother suggested that I should consult a doctor'
complex_sent = 'John retired when he turned 65'
complex_sent2 = 'Whether you agree with me or not makes little \
difference to our inverstors, who by the way, are the ones most \
affected by whatever mistake we make.'
complex_sent3 = 'Whoever thought of the idea is a genius.'
complex_compound1 = "Bill voted against the measure \
because he felt that it wasn't strong enough, but he also offered \
to continue discussions, which we will do next week." 
current = sentence2
doc = nlp(current)
print("The sentence :", current," \n\n")

print("Its analysis :")
for token in doc:
    print(token.text," | ",  token.dep_," | ", token.pos_, " | ", token.head.text," | ",  token.head.pos_," | ", 
           [left.text for left in token.lefts], " | ",
           [right.text for right in token.rights], " | ",
           [child.text for child in token.children])
    
print("\n\n")

'''
black_list = ["det", "punct", "conj", "cconj"]
clausal_list = ["csubj", 'ccomp', 
                'advcl', 'acl', 'conj',
                'relcl']

print(extract_clauses(current))

extracted_clauses = extract_clauses(current)
for clause in extracted_clauses:
    print("Clause :", clause, check_verb_subject_in_clause(clause))

print("Printing the extracted clauses :", extracted_clauses)
print("\n\n")
entities_other = get_entities(current)
relation_other = get_relation(current)
print("\nentity and relations from the blog:\nSubject : {} | Object : {} | Relation : {}\n\n".format(entities_other[0], 
                                                                               entities_other[1],
                                                                               relation_other))

entity_list = extract_entity_relation_tuples(current)
for entity in entity_list:
    subj, obj = entity[0]
    rel = entity[1]
    print("\nentity and relations from my code:\nSubject : {} | Object : {} | Relation : {}\n\n".format(subj, 
                                                                              obj, rel))

print(extract_entities_from_clause(current))
print(extract_relation_from_clause(current))
'''   
displacy.serve(doc, style='dep')
    

The sentence : Even with the weather being that nasty, the couple and their families decided to go ahead with the wedding as planned.  


Its analysis :
Even  |  advmod  |  ADV  |  with  |  ADP  |  []  |  []  |  []
with  |  prep  |  ADP  |  decided  |  VERB  |  ['Even']  |  ['being']  |  ['Even', 'being']
the  |  det  |  DET  |  weather  |  NOUN  |  []  |  []  |  []
weather  |  nsubj  |  NOUN  |  being  |  AUX  |  ['the']  |  []  |  ['the']
being  |  pcomp  |  AUX  |  with  |  ADP  |  ['weather']  |  ['nasty']  |  ['weather', 'nasty']
that  |  advmod  |  ADV  |  nasty  |  ADJ  |  []  |  []  |  []
nasty  |  acomp  |  ADJ  |  being  |  AUX  |  ['that']  |  []  |  ['that']
,  |  punct  |  PUNCT  |  decided  |  VERB  |  []  |  []  |  []
the  |  det  |  DET  |  couple  |  NOUN  |  []  |  []  |  []
couple  |  nsubj  |  NOUN  |  decided  |  VERB  |  ['the']  |  ['and', 'families']  |  ['the', 'and', 'families']
and  |  cc  |  CCONJ  |  couple  |  NOUN  |  []  |  []  |  []
their  |  poss  |  D

/usr/lib/python3.6/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [29/May/2020 13:12:44] "GET / HTTP/1.1" 200 17617
127.0.0.1 - - [29/May/2020 13:12:44] "GET /favicon.ico HTTP/1.1" 200 17617


In [14]:
print("The sentence : {} \nThe extraction : {} \nThe other extraction : {} \n\
The relation : |{}".format(current, entities, entities_other, relation_other))

The sentence : Her story might be embarrasing for some of her friends. 
The extraction : ('Her story', 'some') 
The other extraction : ['story', 'friends'] 
The relation : |be embarrasing


In [24]:
sent = complex_sent
doc = nlp(sent)

for token in doc:
    print("The token : {} | The subtree : \n".format(token))
    for sub in token.subtree:
        print(sub.lefts)

The token : John | The subtree : 

The token : retired | The subtree : 

The token : when | The subtree : 

The token : he | The subtree : 

The token : turned | The subtree : 

The token : 65 | The subtree : 



In [14]:
doc

John retired when he turned 65